In [1]:
import QUANTAXIS as QA

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.1.0
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##`````##```##```````##```````##````##```````###````````##``````###```` 
  ##````````##```##````````##`````##``````

# 在这里我们演示一下 下单/交易/结算的整个流程

我们首先会建立一个账户类和一个回测类


In [2]:
 # 初始化一个account
Account=QA.QA_Account()

# 初始化一个回测类
B = QA.QA_BacktestBroker()


在第一天的时候,全仓买入 000001

In [3]:
# 全仓买入'000001'

Order=Account.send_order(code='000001',
                        price=11,
                        money=0.04*Account.cash_available,
                        time='2018-05-09',
                        towards=QA.ORDER_DIRECTION.BUY,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_MONEY
                        )




In [4]:

## 打印order的占用资金
print('ORDER的占用资金: {}'.format((Order.amount*Order.price)*(1+Account.commission_coeff)))

ORDER的占用资金: 39609.9


In [5]:
# 账户剩余资金
print('账户剩余资金 :{}'.format(Account.cash_available))

账户剩余资金 :960000.0


In [6]:
Account.hold

Series([], Name: amount, dtype: float64)

In [7]:
Account.init_hold

Series([], Name: amount, dtype: float64)

In [8]:
Account.hold_available

Series([], Name: amount, dtype: float64)

此时的账户cash并未减少,因为此过程为申报订单(已委托 未成交状态)

回测类接受订单 并返回订单(如果创建成功)

In [9]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

< QA_Order realorder_id Order_T4Fv8xm3 datetime:2018-05-09 09:30:00 code:000001 amount:3600 price:10.955 towards:1 btype:stock_cn order_id:Order_T4Fv8xm3 account:Acc_4yMihLnm status:100 >


In [10]:
#B.query_orders(rec_mes.account_cookie,rec_mes.realorder_id)

In [11]:
import pandas as pd

In [13]:
B.query_orders(Account.account_cookie)

,,order_time,code,name,towards,trade_price,order_price,status,order_amount,trade_amount,cancel_amount
account_cookie,realorder_id,,,,,,,,,,
Acc_4yMihLnm,Trade_6Fzyb5aJ,2018-05-09 09:31:00,000001,None,1,10.96,10.955,100,3600,3600,0


In [ ]:
pd.DataFrame(list(B.deal_message.values()),columns=B.orderstatus_headers).set_index(['account_cookie','realorder_id'])

In [ ]:
B.query_deal

账户类接收到回测返回的回报信息,更新账户

In [ ]:
Account.receive_deal(rec_mes)

此时我们可以打印一下现在的状态(现在的状态可以理解为在交易时 买入一只000001股票,但当天尚未收盘)

In [ ]:
print('账户的可用资金 {}'.format(Account.cash_available))

In [ ]:
Account.hold

In [ ]:
Account.hold_available

In [ ]:
Account.init_hold.index_name='code'

In [ ]:
import pandas as pd

In [ ]:
pd.concat([Account.hold_available,Account.init_hold])

In [ ]:
+Account.hold_available

我们注意到 当最初申报订单的时候,可用资金只有950.2999999999302元,而买入成功后,可用资金有3339.9289999998837元,原因是下单的时候模式是市价单模式(QA.ORDER_MODEL.MARKET),故实际成交金额为10.96元

买入以后 账户的持仓为90800股 000001

In [ ]:
Account.hold

买入后账户现金表被扩展

In [ ]:
Account.cash

因为是t+1的A股市场,故此时可卖数量为0

In [ ]:
Account.sell_available

# 执行结算

In [ ]:
Account.settle()

# 结算后

In [ ]:
Account.cash

In [ ]:
Account.cash_available

In [ ]:
Account.sell_available

In [ ]:
Account.hold

# 执行卖出操作

现在的持仓为: 000001 90800股

In [ ]:
holdnum=Account.sell_available.get('000001',0)

In [ ]:
holdnum

申报一个卖出单,把可卖全部卖出

In [ ]:
Order=Account.send_order(code='000001',
                        price=11,
                        amount=holdnum,
                        time='2018-05-10',
                        towards=QA.ORDER_DIRECTION.SELL,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_AMOUNT
                        )


In [ ]:
Order

In [ ]:
Account.cash_available  # 因为此时订单尚未申报成功 可用现金不变

In [ ]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

In [ ]:
Account.receive_deal(rec_mes)

In [ ]:
Account.cash_available # 此时订单已成交 cash_available立刻结转

In [ ]:
Account.history_table

In [ ]:
Account.orders

In [ ]:
Account.orders.queue_df.query('date=="2018-05-09"')

# 测试T0账户

In [ ]:
 # 初始化一个account
AccountT0=QA.QA_Account(running_environment=QA.RUNNING_ENVIRONMENT.TZERO,init_hold={'000001':10000},init_cash=200000)

# 初始化一个回测类
B = QA.QA_BacktestBroker()

In [ ]:
AccountT0.init_assets

In [ ]:
AccountT0.init_hold

In [ ]:
AccountT0.hold_available

In [ ]:
AccountT0.sell_available

In [ ]:
Order=AccountT0.send_order(code='000001',
                        price=11,
                        amount=AccountT0.sell_available.get('000001',0),
                        time='2018-05-10',
                        towards=QA.ORDER_DIRECTION.SELL,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_AMOUNT
                        )


In [ ]:
Order

In [ ]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

In [ ]:
AccountT0.receive_deal(rec_mes)

In [ ]:
AccountT0.sell_available

In [ ]:
AccountT0.hold

In [ ]:
AccountT0.buy_available

In [ ]:
AccountT0.hold_available

In [ ]:
AccountT0.running_time

In [ ]:
AccountT0.datetime

In [ ]:
#AccountT0.close_positions_order

In [ ]:
for item in AccountT0.close_positions_order:
    rec_mes=B.receive_order(QA.QA_Event(order=item))
    print(rec_mes)
    AccountT0.receive_deal(rec_mes)

In [ ]:
AccountT0.cash

In [ ]:
AccountT0.sell_available

In [ ]:
AccountT0.hold_available

In [ ]:
AccountT0.hold

In [ ]:
AccountT0.settle()

In [ ]:
AccountT0.daily_hold

In [ ]:
AccountT0.daily_cash

In [ ]:
AccountT0.hold_table()

In [ ]:
AccountT0.hold_price()

In [ ]:
AccountT0.datetime

In [ ]:
AccountT0.sell_available

In [ ]:
risk_t0=QA.QA_Risk(AccountT0)

In [ ]:
risk_t0.init_assets

In [ ]:
risk_t0.init_cash-risk_t0.assets

In [ ]:
risk_t0.assets.iloc[-1]/risk_t0.init_cash-1

In [ ]:
risk_t0.profit_construct